In [1]:
import dspy
from dspy.predict import Retry
from dspy.primitives.assertions import assert_transform_module, backtrack_handler
import importlib

importlib.reload(dspy)

# Configure the AzureOpenAI language model
azure_turbo = dspy.AzureOpenAI(api_base='https://bionlp-gpt4-wang.openai.azure.com/', api_key="a494edc84d714b6c8a12e7212974b793", api_version='2024-03-01-preview', model='gpt-35-turbo', max_tokens=400)
dspy.settings.configure(lm=azure_turbo)

/opt/anaconda3/envs/dspy_repo/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import re

def eval_metric(true, prediction, trace=None):
    try:
        pred = prediction.answer
        matches = re.findall(r"\([A-Z]\)", pred)
        parsed_answer = matches[-1] if matches else ""
        return parsed_answer == true.answer
    except:
        return False 

In [3]:
class GenerateAnswer(dspy.Signature):
    """You are a helpful medical expert, and your task is to answer a multi-choice medical question. Please first think step-by-step and then choose the answer from the provided options."""
    
    question = dspy.InputField(desc="Question")
    options = dspy.InputField(desc="Answer choice options")
    answer = dspy.OutputField(desc="You should respond with one of (A), (B), (C), or (D)")

In [4]:
class AnswerQuestion(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
    
    def forward(self, question, options):
        response = self.generate_answer(question=question, options=options)
      
        #valid_response = "(A)" in response.answer or "(B)" in response.answer or "(C)" in response.answer or "(D)" in response.answer  

        #dspy.Assert(valid_response, "You must respond with one of (A), (B), (C), or (D) as part of your answer.")

        return dspy.Prediction(answer=response.answer)

In [1]:
import json

train_data = []
val_data = []
test_data = []

with open("/Users/khandekarns/Downloads/data_clean/questions/US/4_options/phrases_no_exclude_train.jsonl") as file:
    for line in file:
        qa_info = json.loads(line)
        train_data.append(qa_info)
        

with open("/Users/khandekarns/Downloads/data_clean/questions/US/4_options/phrases_no_exclude_dev.jsonl") as file:
    for line in file:
        qa_info = json.loads(line)
        val_data.append(qa_info)


with open("/Users/khandekarns/Downloads/data_clean/questions/US/4_options/phrases_no_exclude_test.jsonl") as file:
    for line in file:
        qa_info = json.loads(line)
        test_data.append(qa_info)

In [2]:
import json


def get_instances(file_path):

    items = []

    with open(file_path) as file:
        indices = json.load(file)

    if "train" in file_path:
        dataset = train_data

    if "test" in file_path:
        dataset = test_data 

    if "val" in file_path:
        dataset = val_data 
    
    for index in indices:
        index = int(index)

        items.append(dataset[index])
    
    return items


train_data = get_instances("/Users/khandekarns/Documents/DSPy/eligible_notes/medqa_train_eligible.json")
val_data = get_instances("/Users/khandekarns/Documents/DSPy/eligible_notes/medqa_val_eligible.json")
test_data = get_instances("/Users/khandekarns/Documents/DSPy/eligible_notes/medqa_test_eligible.json")

In [4]:
# Only pass in the valid ones and save this to a jsonl file 

with open("/Users/khandekarns/Documents/MedQA_GPT35/train.jsonl", 'w') as file:
    for dictionary in train_data:
        # Convert the dictionary to a JSON string and write it to the file
        json_line = json.dumps(dictionary)
        file.write(json_line + '\n')


with open("/Users/khandekarns/Documents/MedQA_GPT35/val.jsonl", 'w') as file:
    for dictionary in val_data:
        # Convert the dictionary to a JSON string and write it to the file
        json_line = json.dumps(dictionary)
        file.write(json_line + '\n')


with open("/Users/khandekarns/Documents/MedQA_GPT35/test.jsonl", 'w') as file:
    for dictionary in test_data:
        # Convert the dictionary to a JSON string and write it to the file
        json_line = json.dumps(dictionary)
        file.write(json_line + '\n')

In [7]:
def question_with_options(options):

   return "\n(A) " + options['A'] + "\n(B) " + options['B'] +  "\n(C) " + options['C'] + "\n(D) " + options['D']


def generate_dspy_examples(dataset):

   examples = []
   
   for i in range(len(dataset)):
      options = question_with_options(dataset[i]['options'])
      
      example = dspy.Example({"question": dataset[i]['question'], "options": options, "answer": "(" + dataset[i]["answer_idx"] + ")"}).with_inputs("question", "options") 

      examples.append(example)

   return examples 


train_dspy_examples = generate_dspy_examples(train_data)
val_dspy_examples = generate_dspy_examples(val_data)
test_dspy_examples = generate_dspy_examples(test_data)

In [8]:
from dspy.evaluate import Evaluate

evaluate_test = Evaluate(devset=test_dspy_examples, metric=eval_metric, num_threads=6, display_progress=True, display_table=True)

In [9]:
evaluate_test(AnswerQuestion())

Average Metric: 787 / 1270  (62.0): 100%|██████████| 1270/1270 [00:03<00:00, 408.20it/s]


,question,options,example_answer,pred_answer,eval_metric
0,A 23-year-old woman comes to the physician because she is embarrassed about the appearance of her nails. She has no history of serious illness and...,(A) Silvery plaques on extensor surfaces (B) Flesh-colored papules in the lumbosacral region (C) Erosions of the dental enamel (D) Holosystolic murmur at the left...,(A),None of the above,False
1,"A junior orthopaedic surgery resident is completing a carpal tunnel repair with the department chairman as the attending physician. During the case, the resident inadvertently...",(A) Disclose the error to the patient and put it in the operative report (B) Tell the attending that he cannot fail to disclose this...,(B),(A) Disclose the error to the patient and put it in the operative report,False
2,A 39-year-old man presents to the emergency department because of progressively worsening chest pain and nausea that started at a local bar 30 minutes prior....,(A) Diltiazem (B) Labetalol (C) Propranolol (D) Reassurance and continuous monitoring,(D),(C) Propranolol,False
3,A 65-year-old man is brought to the emergency department 30 minutes after the onset of acute chest pain. He has hypertension and asthma. Current medications...,(A) Nifedipine (B) Enoxaparin (C) Clopidogrel (D) Spironolactone,(C),(B) Enoxaparin,False
4,"A 72-year-old man comes to the physician because of a 2-month history of fatigue and worsening abdominal pain. During this period, he also has excessive...",(A) Cladribine (B) Prednisone (C) Imatinib (D) Ruxolitinib,(D),(D) Ruxolitinib,✔️ [True]
5,A 37-year-old-woman presents to her primary care physician requesting a new form of birth control. She has been utilizing oral contraceptive pills (OCPs) for the...,(A) A history of stroke or venous thromboembolism (B) Active or recurrent pelvic inflammatory disease (PID) (C) Past medical history of breast cancer (D) Known...,(B),(B) Active or recurrent pelvic inflammatory disease (PID),✔️ [True]
6,A 68-year-old male comes to the physician for evaluation of right flank pain. He has a history of diabetes and peripheral artery disease. His blood...,(A) Renal artery stenosis (B) Benign prostatic hyperplasia (C) Common iliac artery aneurysm (D) Urethral stricture,(C),(A) Renal artery stenosis,False
7,A 67-year-old man with transitional cell carcinoma of the bladder comes to the physician because of a 2-day history of ringing sensation in his ear....,(A) Inhibition of proteasome (B) Hyperstabilization of microtubules (C) Generation of free radicals (D) Cross-linking of DNA,(D),(C) Generation of free radicals,False
8,A 24-year-old G2P1 woman at 39 weeks’ gestation presents to the emergency department complaining of painful contractions occurring every 10 minutes for the past 2...,"(A) It is a Southwestern blot, identifying the presence of DNA-binding proteins (B) It is a Northern blot, identifying the presence of RNA (C) It...",(D),(D) It is an HIV-1/HIV2 antibody differentiation immunoassay,✔️ [True]
9,A 20-year-old man comes to the physician because of worsening gait unsteadiness and bilateral hearing loss for 1 month. He has had intermittent tingling sensations...,(A) Renal cell carcinoma (B) Meningioma (C) Astrocytoma (D) Vascular malformations,(B),(B) Meningioma,✔️ [True]


61.97

In [14]:
from dspy.teleprompt import MIPROv2
import random

random.seed(42)

#train_dspy_samples = random.sample(train_dspy_examples, 3000)
#val_dspy_samples = random.sample(val_dspy_examples, 300)

teleprompter = MIPROv2(prompt_model= azure_turbo, task_model=azure_turbo, metric=eval_metric, num_candidates=20, metric_threshold=None)
kwargs = dict(num_threads=3, display_progress=True, display_table=True)
compiled_prompt_opt = teleprompter.compile(AnswerQuestion(), trainset=train_dspy_examples, valset= val_dspy_examples, max_bootstrapped_demos=5, eval_kwargs=kwargs, requires_permission_to_run=False)


Please be advised that based on the parameters you have set, the maximum number of LM calls is projected as follows:


- Prompt Model: 10 data summarizer calls + 20 * 1 lm calls in program + (2) lm calls in program aware proposer = 32 prompt model calls
- Task Model: 25 examples in minibatch * 30 batches + 10146 examples in train set * 3 full evals = 31188 task model calls

Estimated Cost Calculation:

Total Cost = (Number of calls to task model * (Avg Input Token Length per Call * Task Model Price per Input Token + Avg Output Token Length per Call * Task Model Price per Output Token) 
            + (Number of calls to prompt model * (Avg Input Token Length per Call * Task Prompt Price per Input Token + Avg Output Token Length per Call * Prompt Model Price per Output Token).

For a preliminary estimate of potential costs, we recommend you perform your own calculations based on the task
and prompt models you intend to use. If the projected costs exceed your budget or expectations, you 

INFO:backoff:Backing off request(...) for 0.6s (openai.BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4096 tokens. However, you requested 4430 tokens (4030 in the messages, 400 in the completion). Please reduce the length of the messages or completion.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}})


Backing off 0.6 seconds after 1 tries calling function <function AzureOpenAI.request at 0x116d52b60> with kwargs {'n': 1, 'temperature': 1.0}


INFO:backoff:Backing off request(...) for 0.1s (openai.BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4096 tokens. However, you requested 4430 tokens (4030 in the messages, 400 in the completion). Please reduce the length of the messages or completion.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}})
INFO:backoff:Backing off request(...) for 1.5s (openai.BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4096 tokens. However, you requested 4430 tokens (4030 in the messages, 400 in the completion). Please reduce the length of the messages or completion.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}})


Backing off 0.1 seconds after 2 tries calling function <function AzureOpenAI.request at 0x116d52b60> with kwargs {'n': 1, 'temperature': 1.0}
Backing off 1.5 seconds after 3 tries calling function <function AzureOpenAI.request at 0x116d52b60> with kwargs {'n': 1, 'temperature': 1.0}


...

FULL TRACE


Average Metric: 0 / 1  (0.0): 100%|██████████| 1/1 [00:02<00:00,  2.11s/it]


,question,options,example_answer,pred_answer,eval_metric
0,"A 23-year-old man presents with a blunt force injury to the head from a baseball bat. He is currently unconscious, although his friends say he...",(A) Mannitol (B) Maintain a PaCO2 of 24 mm Hg (C) Placement of a ventriculoperitoneal (VP) shunt (D) Acetazolamide,(A),None of the options provided are the most appropriate medical treatment for this patient.,False


[I 2024-07-23 21:32:29,358] Trial 29 finished with value: 64.0 and parameters: {'0_predictor_instruction': 11, '0_predictor_demos': 13}. Best is trial 25 with value: 92.0.





You are a medical expert and your task is to answer a multiple-choice medical question by providing a step-by-step rationale. Read the question carefully and consider each option before selecting the answer. Take into account the patient's history, physical examination findings, and any relevant medical knowledge or guidelines. Provide a detailed reasoning for your answer. Remember to refer to the specific milestones, symptoms, or mechanisms mentioned in the question to support your choice. Use your clinical reasoning skills to select the most appropriate answer. Finally, respond with the letter (A), (B), (C), or (D) corresponding to your selected answer.

---

Follow the following format.

Question: Question

Options: Answer choice options

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: You should respond with one of (A), (B), (C), or (D)

---

Question: A patient is infected with a pathogen and produces many antibodies to many antigens assoc

In [15]:
evaluate_test(compiled_prompt_opt)

Average Metric: 40 / 59  (67.8):   5%|▍         | 59/1270 [00:19<08:02,  2.51it/s]INFO:backoff:Backing off request(...) for 0.8s (openai.RateLimitError: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-03-01-preview have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 1 second. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}})


Backing off 0.8 seconds after 1 tries calling function <function AzureOpenAI.request at 0x116d52b60> with kwargs {}


Average Metric: 42 / 61  (68.9):   5%|▍         | 61/1270 [00:21<13:35,  1.48it/s]

Backing off 0.0 seconds after 1 tries calling function <function AzureOpenAI.request at 0x116d52b60> with kwargs {}


Average Metric: 42 / 64  (65.6):   5%|▌         | 64/1270 [00:22<10:25,  1.93it/s]INFO:backoff:Backing off request(...) for 0.7s (openai.RateLimitError: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-03-01-preview have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 1 second. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}})


...

Average Metric: 857 / 1243  (68.9):  98%|█████████▊| 1242/1270 [09:57<00:19,  1.46it/s]INFO:backoff:Backing off request(...) for 0.1s (openai.RateLimitError: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-03-01-preview have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 1 second. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}})


Backing off 0.1 seconds after 2 tries calling function <function AzureOpenAI.request at 0x116d52b60> with kwargs {}


Average Metric: 875 / 1270  (68.9): 100%|██████████| 1270/1270 [10:11<00:00,  2.08it/s]


,question,options,example_answer,pred_answer,eval_metric
0,A 23-year-old woman comes to the physician because she is embarrassed about the appearance of her nails. She has no history of serious illness and...,(A) Silvery plaques on extensor surfaces (B) Flesh-colored papules in the lumbosacral region (C) Erosions of the dental enamel (D) Holosystolic murmur at the left...,(A),(A) Silvery plaques on extensor surfaces,✔️ [True]
1,"A junior orthopaedic surgery resident is completing a carpal tunnel repair with the department chairman as the attending physician. During the case, the resident inadvertently...",(A) Disclose the error to the patient and put it in the operative report (B) Tell the attending that he cannot fail to disclose this...,(B),(A) Disclose the error to the patient and put it in the operative report,False
2,A 39-year-old man presents to the emergency department because of progressively worsening chest pain and nausea that started at a local bar 30 minutes prior....,(A) Diltiazem (B) Labetalol (C) Propranolol (D) Reassurance and continuous monitoring,(D),(D) Reassurance and continuous monitoring,✔️ [True]
3,A 65-year-old man is brought to the emergency department 30 minutes after the onset of acute chest pain. He has hypertension and asthma. Current medications...,(A) Nifedipine (B) Enoxaparin (C) Clopidogrel (D) Spironolactone,(C),(C) Clopidogrel,✔️ [True]
4,"A 72-year-old man comes to the physician because of a 2-month history of fatigue and worsening abdominal pain. During this period, he also has excessive...",(A) Cladribine (B) Prednisone (C) Imatinib (D) Ruxolitinib,(D),(D) Ruxolitinib,✔️ [True]
5,A 37-year-old-woman presents to her primary care physician requesting a new form of birth control. She has been utilizing oral contraceptive pills (OCPs) for the...,(A) A history of stroke or venous thromboembolism (B) Active or recurrent pelvic inflammatory disease (PID) (C) Past medical history of breast cancer (D) Known...,(B),(D) Known liver neoplasm,False
6,A 68-year-old male comes to the physician for evaluation of right flank pain. He has a history of diabetes and peripheral artery disease. His blood...,(A) Renal artery stenosis (B) Benign prostatic hyperplasia (C) Common iliac artery aneurysm (D) Urethral stricture,(C),(C) Common iliac artery aneurysm,✔️ [True]
7,A 67-year-old man with transitional cell carcinoma of the bladder comes to the physician because of a 2-day history of ringing sensation in his ear....,(A) Inhibition of proteasome (B) Hyperstabilization of microtubules (C) Generation of free radicals (D) Cross-linking of DNA,(D),(C) Generation of free radicals,False
8,A 24-year-old G2P1 woman at 39 weeks’ gestation presents to the emergency department complaining of painful contractions occurring every 10 minutes for the past 2...,"(A) It is a Southwestern blot, identifying the presence of DNA-binding proteins (B) It is a Northern blot, identifying the presence of RNA (C) It...",(D),(D) It is an HIV-1/HIV2 antibody differentiation immunoassay,✔️ [True]
9,A 20-year-old man comes to the physician because of worsening gait unsteadiness and bilateral hearing loss for 1 month. He has had intermittent tingling sensations...,(A) Renal cell carcinoma (B) Meningioma (C) Astrocytoma (D) Vascular malformations,(B),(B) Meningioma,✔️ [True]


68.9